# Data cleaning project for a SQLite Database

The following project is intended to clean a Dataset in order to introduce it to a SQLite database (db). Specifically this project will make sure that all entries will have the same format to allow for proper querying in a db.

## The Dataset

The Dataset that will be used for this project contains information about the the Academy Award Winners and Nominees from 1928 till 2010. This dataset will be contained in the "academy_awards.csv" file.

The columns are as follow:

1) Year: year(s) of the award ceremony.

2) Category: category of the award the nominee was nominated for.

3) Nominee: person nominated for the award.

4) Aditional Info: cotains things like:

- the movie the nominee participated in.

- the character the nominee played.

5) Won?: contains yes and no to express if it won.

## Cleaning the Dataset

In [1]:
import pandas as pd
import sqlite3

In [2]:
df = pd.read_csv('academy_awards.csv', encoding= "ISO-8859-1")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10137 entries, 0 to 10136
Data columns (total 11 columns):
Year               10137 non-null object
Category           10137 non-null object
Nominee            10137 non-null object
Additional Info    9011 non-null object
Won?               10137 non-null object
Unnamed: 5         11 non-null object
Unnamed: 6         12 non-null object
Unnamed: 7         3 non-null object
Unnamed: 8         2 non-null object
Unnamed: 9         1 non-null object
Unnamed: 10        1 non-null object
dtypes: object(11)
memory usage: 871.2+ KB


### The Unnamed Columns

In [3]:
df[[i for i in df.columns if 'Unnamed' in i]].isnull().sum()

Unnamed: 5     10126
Unnamed: 6     10125
Unnamed: 7     10134
Unnamed: 8     10135
Unnamed: 9     10136
Unnamed: 10    10136
dtype: int64

These columns will be eliminated, given that they are mostly compromised of NaN values.

In [4]:
df.drop(['Unnamed: ' + str(i) for i in range(5,11)], axis= 1, inplace = True)
df.head()

,Year,Category,Nominee,Additional Info,Won?
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO
3,2010 (83rd),Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES
4,2010 (83rd),Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO


### The Year Column

Having eliminated the columns that provided  little information for the Dataset, it is time to focus on the Year column.

In [5]:
df.Year.value_counts()

1941 (14th)      192
1942 (15th)      190
1943 (16th)      188
1944 (17th)      181
1945 (18th)      163
1940 (13th)      162
1939 (12th)      153
1998 (71st)      150
1938 (11th)      145
1954 (27th)      141
1995 (68th)      141
2009 (82nd)      140
1952 (25th)      138
1994 (67th)      138
2001 (74th)      137
1997 (70th)      137
2010 (83rd)      135
1953 (26th)      135
1937 (10th)      135
1956 (29th)      134
1964 (37th)      132
1955 (28th)      132
1996 (69th)      131
2006 (79th)      130
2000 (73rd)      129
2004 (77th)      129
1961 (34th)      129
1990 (63rd)      129
1966 (39th)      129
2005 (78th)      129
                ... 
1993 (66th)      119
2008 (81st)      118
1975 (48th)      118
1985 (58th)      116
1950 (23rd)      116
1983 (56th)      116
1974 (47th)      115
1948 (21st)      115
1946 (19th)      115
1936 (9th)       115
1958 (31st)      115
1989 (62nd)      114
1972 (45th)      114
1947 (20th)      114
1980 (53rd)      113
1967 (40th)      113
1976 (49th)  

The years columns is comprised of the year(s) in which the ceremony nominated an actor or actress. It appears that the first years of the ceremony honored the nominees from a period of two years (this explains the "/" character). This column also specifies the number of the ceremony. Both facts must be taken into consideration:

1) To avoid data redundancy, in the case of ceremonies that honored nominees from a period of years, only the first year will be considered.

2) A 'ceremony_id' column will be added to consider the number of each ceremony.

In [6]:
df['ceremony_id'] = [i[i.find('(')+1:i.find(')')-2] for i in df.Year]
df.ceremony_id.head()

0    83
1    83
2    83
3    83
4    83
Name: ceremony_id, dtype: object

In [7]:
df['Year'] = [i[:4] for i in df.Year]
df.head()

,Year,Category,Nominee,Additional Info,Won?,ceremony_id
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,83
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,83
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,83
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,83
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,83


### The Category column

The Category column has the role for which actor/actress was nominated for.

In [8]:
df.Category.value_counts()

Writing                                                        888
Music (Scoring)                                                748
Cinematography                                                 572
Art Direction                                                  552
Best Picture                                                   485
Sound                                                          460
Short Film (Live Action)                                       434
Scientific and Technical (Technical Achievement Award)         428
Music (Song)                                                   413
Actress -- Leading Role                                        411
Directing                                                      410
Actor -- Leading Role                                          408
Film Editing                                                   385
Costume Design                                                 384
Actor -- Supporting Role                                      

In [9]:
df[df['Category'] == 'Best Picture'].head()

,Year,Category,Nominee,Additional Info,Won?,ceremony_id
75,2010,Best Picture,Black Swan,"Mike Medavoy, Brian Oliver and Scott Franklin,...",NO,83
76,2010,Best Picture,The Fighter,"David Hoberman, Todd Lieberman and Mark Wahlbe...",NO,83
77,2010,Best Picture,Inception,"Emma Thomas and Christopher Nolan, Producers",NO,83
78,2010,Best Picture,The Kids Are All Right,"Gary Gilbert, Jeffrey Levy-Hinte and Celine Ra...",NO,83
79,2010,Best Picture,The King's Speech,"Iain Canning, Emile Sherman and Gareth Unwin, ...",YES,83


It appears that the Category column include awards that go beyond Actors; it also includes awards for movies and others. This implies that it might be better to break off the dataset when it comes time to add it to a db, given that the Additional Info column will contain different information.

In [10]:
for i in df.Category.unique():
    print(df[df.Category == i][['Category', 'Additional Info']].iloc[0])

Category           Actor -- Leading Role
Additional Info       Biutiful {'Uxbal'}
Name: 0, dtype: object
Category               Actor -- Supporting Role
Additional Info    The Fighter {'Dicky Eklund'}
Name: 5, dtype: object
Category                  Actress -- Leading Role
Additional Info    The Kids Are All Right {'Nic'}
Name: 10, dtype: object
Category                 Actress -- Supporting Role
Additional Info    The Fighter {'Charlene Fleming'}
Name: 15, dtype: object
Category                    Animated Feature Film
Additional Info    Chris Sanders and Dean DeBlois
Name: 20, dtype: object
Category                                               Art Direction
Additional Info    Production Design: Robert Stromberg; Set Decor...
Name: 23, dtype: object
Category              Cinematography
Additional Info    Matthew Libatique
Name: 28, dtype: object
Category           Costume Design
Additional Info    Colleen Atwood
Name: 33, dtype: object
Category                  Directing
Additional I

Given that there are many categories for actors and actress one table could include all of these, and their respective information.

In [11]:
act_rows = [i for i in df.Category.unique() if "Act" in i]
act_rows

['Actor -- Leading Role',
 'Actor -- Supporting Role',
 'Actress -- Leading Role',
 'Actress -- Supporting Role',
 'Short Film (Live Action)',
 'Acting (other)']

In [12]:
act_rows.remove('Short Film (Live Action)')
act_rows

['Actor -- Leading Role',
 'Actor -- Supporting Role',
 'Actress -- Leading Role',
 'Actress -- Supporting Role',
 'Acting (other)']

In [13]:
actors = df[df.Category.isin(act_rows)].copy()
actors.head()

,Year,Category,Nominee,Additional Info,Won?,ceremony_id
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,83
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,83
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,83
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,83
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,83


Having isolated all awards nominee for actors, now it's time to focus on the Additional Info column which provides information on the movie each actor/actress participated in and the role they had. But first it is important to eliminate the NaN values.

In [14]:
actors['Additional Info'].isnull().sum()

15

Given the small amount of NaNs, these rows will be eliminated.

In [15]:
actors.dropna(subset = ['Additional Info'], axis = 0, inplace = True)
actors.isnull().sum()

Year               0
Category           0
Nominee            0
Additional Info    0
Won?               0
ceremony_id        0
dtype: int64

In [16]:
actors['movies'] = [i.split(' {')[0] for i in actors['Additional Info']]
actors['role'] = [i[i.find('{')+2:i.find('}')-1] for i in actors['Additional Info']]
actors.head()

,Year,Category,Nominee,Additional Info,Won?,ceremony_id,movies,role
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,83,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,83,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,83,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,83,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,83,127 Hours,Aron Ralston


In [17]:
tech_rows = [i for i in df.Category.unique() if i not in act_rows]
tech_rows

['Animated Feature Film',
 'Art Direction',
 'Cinematography',
 'Costume Design',
 'Directing',
 'Documentary (Feature)',
 'Documentary (Short Subject)',
 'Film Editing',
 'Foreign Language Film',
 'Makeup',
 'Music (Scoring)',
 'Music (Song)',
 'Best Picture',
 'Short Film (Animated)',
 'Short Film (Live Action)',
 'Sound',
 'Sound Editing',
 'Visual Effects',
 'Writing',
 'Honorary Award',
 'Irving G. Thalberg Memorial Award',
 'Scientific and Technical (Scientific and Engineering Award)',
 'Scientific and Technical (Technical Achievement Award)',
 'Scientific and Technical (Bonner Medal)',
 'Jean Hersholt Humanitarian Award',
 'Scientific and Technical (Gordon E. Sawyer Award)',
 'Scientific and Technical (Academy Award of Merit)',
 'Scientific and Technical (Special Awards)',
 'Special Achievement Award',
 'Special Effects (archaic category)',
 'Documentary (other)',
 'Assistant Director (archaic category)',
 'Dance Direction (archaic category)',
 'Engineering Effects (archaic cate

In [18]:
tech = df[df.Category.isin(tech_rows)].copy()
tech.head()

,Year,Category,Nominee,Additional Info,Won?,ceremony_id
20,2010,Animated Feature Film,How to Train Your Dragon,Chris Sanders and Dean DeBlois,NO,83
21,2010,Animated Feature Film,The Illusionist,Sylvain Chomet,NO,83
22,2010,Animated Feature Film,Toy Story 3,Lee Unkrich,YES,83
23,2010,Art Direction,Alice in Wonderland,Production Design: Robert Stromberg; Set Decor...,YES,83
24,2010,Art Direction,Harry Potter and the Deathly Hallows Part 1,Production Design: Stuart Craig; Set Decoratio...,NO,83


Having split the dataset into two different ones (actors, and tech), there's need to be a new dataset that can link both of thes e. For the time being, this will have information only for the ceremony itself: year and id.

In [19]:
ceremonies = pd.DataFrame({
    'year' : df.Year.unique(),
    'ceremony_id' : df.ceremony_id.unique()
})
ceremonies.head()

,ceremony_id,year
0,83,2010
1,82,2009
2,81,2008
3,80,2007
4,79,2006


In [20]:
nominations = pd.Series(df.groupby('Year').size()).reset_index()

In [21]:
ceremonies = ceremonies.merge(nominations, left_on = 'year', right_on = 'Year')

In [22]:
ceremonies.drop('year', axis = 1, inplace = True)
ceremonies.rename(columns = {0:'nominations'}, inplace = True)
ceremonies.head()

,ceremony_id,Year,nominations
0,83,2010,135
1,82,2009,140
2,81,2008,118
3,80,2007,127
4,79,2006,130


As of now, there are three datasets that come from the original: ceremonies, tech, and actors. These will be the tables that form part of the the db, with the ceremonies table the one that connects them all together.

### The Won? column

The SQLite column does not support boolean values, therefore it would be convenient to transform this column so that it can be queried and aggregated for calculations easier. That is why the values will be changed to 1 (YES) and 0 (NO). All datasets that will belong to the final db, must be formatted like so.

In [23]:
bol_val = {
    'NO' : 0,
    'YES' : 1
}

tech['Won?'] = tech['Won?'].map(bol_val)
actors['Won?'] = actors['Won?'].map(bol_val)
print(tech.head())
print(actors.head())


    Year               Category                                      Nominee  \
20  2010  Animated Feature Film                     How to Train Your Dragon   
21  2010  Animated Feature Film                              The Illusionist   
22  2010  Animated Feature Film                                  Toy Story 3   
23  2010          Art Direction                          Alice in Wonderland   
24  2010          Art Direction  Harry Potter and the Deathly Hallows Part 1   

                                      Additional Info  Won? ceremony_id  
20                     Chris Sanders and Dean DeBlois   0.0          83  
21                                     Sylvain Chomet   0.0          83  
22                                        Lee Unkrich   1.0          83  
23  Production Design: Robert Stromberg; Set Decor...   1.0          83  
24  Production Design: Stuart Craig; Set Decoratio...   0.0          83  
   Year               Category          Nominee  \
0  2010  Actor -- Leadin

Something that might be useful for the future the amount of awards given on every ceremony. An extra column for the ceremonies table (awarded) will sum all of the awards given on that ceremony.

In [24]:
import numpy as np
awards_t = tech.groupby('Year').agg({'Won?': np.sum})
awards_a = actors.groupby('Year').agg({'Won?':np.sum})

print(awards_t.head())
print()
print(awards_a.head())
print(len(awards_a), len(awards_t))

awards = awards_a + awards_t
awards.head()

      Won?
Year      
1927  15.0
1928   5.0
1929   6.0
1930  13.0
1931  13.0

      Won?
Year      
1927     2
1928     2
1929     2
1930     2
1931     3
83 83


,Won?
Year,
1927,17.0
1928,7.0
1929,8.0
1930,15.0
1931,16.0


In [30]:
ceremonies = ceremonies.merge(awards, left_on= 'Year', right_index= True)
ceremonies.rename(columns = {'Won?': 'awards', 'Year':'year'}, inplace = True)
ceremonies.head()

,ceremony_id,year,nominations,awards
0,83,2010,135,38.0
1,82,2009,140,43.0
2,81,2008,118,31.0
3,80,2007,127,36.0
4,79,2006,130,41.0


In [37]:
ceremonies.isnull().sum()
tech.isnull().sum()

Year                  0
Category              0
Nominee               0
Additional Info    1111
Won?                 14
ceremony_id           0
dtype: int64

The tech table has NaN values that must be eliminated.

In [42]:
tech.dropna(inplace = True)
tech.isnull().sum()

Year               0
Category           0
Nominee            0
Additional Info    0
Won?               0
ceremony_id        0
dtype: int64

## Academy Award Database (aadb)

With all tables of the database created, the db must be created and, later, the connections between all tables.

In [43]:
conn = sqlite3.connect('aadb.db')

In [44]:
tech.to_sql('tech', conn, index = False)
ceremonies.to_sql('ceremonies', conn, index = False)
actors.to_sql('actors', conn, index = False)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:1201: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [46]:
q = '''
SELECT *
FROM tech
LIMIT 5
'''
conn.execute(q).fetchall()

[('2010',
  'Animated Feature Film',
  'How to Train Your Dragon',
  'Chris Sanders and Dean DeBlois',
  0.0,
  '83'),
 ('2010',
  'Animated Feature Film',
  'The Illusionist',
  'Sylvain Chomet',
  0.0,
  '83'),
 ('2010', 'Animated Feature Film', 'Toy Story 3', 'Lee Unkrich', 1.0, '83'),
 ('2010',
  'Art Direction',
  'Alice in Wonderland',
  "Production Design: Robert Stromberg; Set Decoration: Karen O'Hara",
  1.0,
  '83'),
 ('2010',
  'Art Direction',
  'Harry Potter and the Deathly Hallows Part 1',
  'Production Design: Stuart Craig; Set Decoration: Stephenie McMillan',
  0.0,
  '83')]